# Clustering Crypto

In [57]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


import warnings
warnings.filterwarnings('ignore')
pd.options.plotting.backend = 'hvplot'

### Fetching Cryptocurrency Data

In [58]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [59]:
# Create a DataFrame 
response = requests.get(url).json()
crypto_df = pd.DataFrame(response['Data']).T
crypto_df.head()


,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,AlgorithmType,Difficulty,BuiltOn,SmartContractAddress,DecimalPoints
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,42,0,0,0,blockchain,scrypt,4.523112,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,300,0,0,0,token,NaN,NaN,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame

### Data Preprocessing

In [61]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df=crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply']]
crypto_df.head(20)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,41.999952,42
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,-1
404,404Coin,Scrypt,True,PoW/PoS,0,-1
433,433 Token,N/A,False,N/A,NaN,NaN
611,SixEleven,SHA-256,True,PoW,0,0
808,808,SHA-256,True,PoW/PoS,0,0
888,Octocoin,N/A,True,PoW,0,0
1337,EliteCoin,X13,True,PoW/PoS,0,0
2015,2015 coin,X11,True,PoW/PoS,0,0


In [62]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df["IsTrading"] == True]
print(crypto_df.shape)
crypto_df.head(20)

(6379, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,41.999952,42
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,-1
404,404Coin,Scrypt,True,PoW/PoS,0,-1
611,SixEleven,SHA-256,True,PoW,0,0
808,808,SHA-256,True,PoW/PoS,0,0
888,Octocoin,N/A,True,PoW,0,0
1337,EliteCoin,X13,True,PoW/PoS,0,0
2015,2015 coin,X11,True,PoW/PoS,0,0
CRAIG,CraigsCoin,X11,True,PoS,NaN,NaN


In [63]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df[crypto_df["Algorithm"] != "N/A"]
print(crypto_df.shape)
crypto_df.head(20)

(1646, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,41.999952,42
365,365Coin,X11,True,PoW/PoS,0,-1
404,404Coin,Scrypt,True,PoW/PoS,0,-1
611,SixEleven,SHA-256,True,PoW,0,0
808,808,SHA-256,True,PoW/PoS,0,0
1337,EliteCoin,X13,True,PoW/PoS,0,0
2015,2015 coin,X11,True,PoW/PoS,0,0
CRAIG,CraigsCoin,X11,True,PoS,NaN,NaN
XBS,Bitstake,X11,True,PoW/PoS,NaN,NaN
XPY,PayCoin,SHA-256,True,PoS,NaN,NaN


In [64]:
# Remove the "IsTrading" column
crypto_df.drop("IsTrading", axis=1, inplace=True)
print(crypto_df.shape)
crypto_df.head(20)

(1646, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,41.999952,42
365,365Coin,X11,PoW/PoS,0,-1
404,404Coin,Scrypt,PoW/PoS,0,-1
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0
1337,EliteCoin,X13,PoW/PoS,0,0
2015,2015 coin,X11,PoW/PoS,0,0
CRAIG,CraigsCoin,X11,PoS,NaN,NaN
XBS,Bitstake,X11,PoW/PoS,NaN,NaN
XPY,PayCoin,SHA-256,PoS,NaN,NaN


In [65]:
# Remove rows with at least 1 null value
crypto_df = crypto_df.dropna(axis=0, how="any")
print(crypto_df.shape)
crypto_df.head(20)

(698, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,41.999952,42
365,365Coin,X11,PoW/PoS,0,-1
404,404Coin,Scrypt,PoW/PoS,0,-1
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0
1337,EliteCoin,X13,PoW/PoS,0,0
2015,2015 coin,X11,PoW/PoS,0,0
XPD,PetroDollar,SHA-256D,N/A,0,-1
ACOIN,ACoin,SHA-256,PoW,0,0
XMY,MyriadCoin,Multiple,PoW,0,2000000000


In [66]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df["TotalCoinsMined"] > 0]
print(crypto_df.shape)
crypto_df.head(20)

(305, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,41.999952,42
NSR,NuShares,PoS,PoS,6173556217.8311,0
TRI,Triangles Coin,X13,PoW/PoS,191620.842403,0
CMTC,CometCoin,Scrypt,PoW,872830,0
CHAT,OpenChat,Scrypt,PoW/PoS,1000000000,-1
QRL,Quantum Resistant Ledger,RandomX,PoW,76031087.968574,105000000
PURA,Pura,X11,PoW,188358976.839698,-1
BTCP,Bitcoin Private,Equihash,PoW,3818878.387802,22873588
ADK,Aidos Kuneen,IMesh,PoW,25000000,0
DAPS,DAPS Coin,Dagger,PoW/PoS/PoA,62319462900,70000000000


In [67]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[crypto_df.iloc[:] != 'N/A'].dropna()
crypto_df.head(20)

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,41.999952,42
NSR,NuShares,PoS,PoS,6173556217.8311,0
TRI,Triangles Coin,X13,PoW/PoS,191620.842403,0
CMTC,CometCoin,Scrypt,PoW,872830,0
CHAT,OpenChat,Scrypt,PoW/PoS,1000000000,-1
QRL,Quantum Resistant Ledger,RandomX,PoW,76031087.968574,105000000
PURA,Pura,X11,PoW,188358976.839698,-1
BTCP,Bitcoin Private,Equihash,PoW,3818878.387802,22873588
ADK,Aidos Kuneen,IMesh,PoW,25000000,0
DAPS,DAPS Coin,Dagger,PoW/PoS/PoA,62319462900,70000000000


In [68]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coinname = pd.DataFrame(crypto_df["CoinName"], index=crypto_df.index)
print(coinname.shape)
coinname.head()

(133, 1)


,CoinName
42,42 Coin
NSR,NuShares
TRI,Triangles Coin
CMTC,CometCoin
CHAT,OpenChat


In [69]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop("CoinName", axis=1)
print(crypto_df.shape)
crypto_df.head(10)

(133, 4)


,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,Scrypt,PoW/PoS,41.999952,42
NSR,PoS,PoS,6173556217.8311,0
TRI,X13,PoW/PoS,191620.842403,0
CMTC,Scrypt,PoW,872830,0
CHAT,Scrypt,PoW/PoS,1000000000,-1
QRL,RandomX,PoW,76031087.968574,105000000
PURA,X11,PoW,188358976.839698,-1
BTCP,Equihash,PoW,3818878.387802,22873588
ADK,IMesh,PoW,25000000,0
DAPS,Dagger,PoW/PoS/PoA,62319462900,70000000000


In [70]:
# Create dummy variables for text features
X = pd.get_dummies(data=crypto_df, columns=["Algorithm", "ProofType"])
print(X.shape)
X.head(20)

(133, 82)


,TotalCoinsMined,MaxSupply,Algorithm_Autolykos,Algorithm_BEP-2,Algorithm_BEP-20 Token,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake2B + SHA3,Algorithm_Blake2b,Algorithm_C31,...,ProofType_PoW/PoSe,ProofType_PoW/nPoS,ProofType_ProgPoW/PoS,ProofType_Proof of Authority,ProofType_Proof-of-Work,ProofType_SPoS,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW,ProofType_dPoW/PoW
42,41.999952,42,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NSR,6173556217.8311,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TRI,191620.842403,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CMTC,872830,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHAT,1000000000,-1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
QRL,76031087.968574,105000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PURA,188358976.839698,-1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BTCP,3818878.387802,22873588,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ADK,25000000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DAPS,62319462900,70000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
# Standardize data
X = StandardScaler().fit_transform(X)
X[:5]

array([[-0.09937731, -0.0932114 , -0.08703883, -0.08703883, -0.08703883,
        -0.12356041, -0.08703883, -0.08703883, -0.12356041, -0.12356041,
        -0.15191091, -0.08703883, -0.08703883, -0.08703883, -0.23570226,
        -0.12356041, -0.08703883, -0.08703883, -0.08703883, -0.3002731 ,
        -0.08703883, -0.08703883, -0.23570226, -0.08703883, -0.08703883,
        -0.12356041, -0.08703883, -0.08703883, -0.08703883, -0.08703883,
        -0.08703883, -0.08703883, -0.15191091, -0.08703883, -0.08703883,
        -0.12356041, -0.19764235, -0.08703883, -0.08703883, -0.12356041,
        -0.12356041, -0.3002731 , -0.12356041, -0.08703883, -0.08703883,
        -0.08703883,  2.18691762, -0.08703883, -0.08703883, -0.08703883,
        -0.08703883, -0.17609018, -0.08703883, -0.19764235, -0.12356041,
        -0.08703883, -0.08703883, -0.08703883, -0.08703883, -0.08703883,
        -0.25298221, -0.08703883, -0.08703883, -0.12356041, -0.12356041,
        -0.08703883, -0.31491833, -0.08703883, -0.0

### Reducing Dimensions Using PCA

In [72]:
# Use PCA to reduce dimensions to 3 principal components
comp_num = 3
pca = PCA(n_components=comp_num)
components = pca.fit_transform(X)
components

array([[-9.04682802e-01, -9.82261606e-01, -2.80979322e-01],
       [-1.28519784e+00,  5.75792549e-01, -6.47673849e-01],
       [-1.48121735e+00, -8.02899235e-01, -3.02798905e-01],
       [ 6.88328530e-01, -8.24449969e-01, -2.36262270e-01],
       [-9.04453407e-01, -9.82033658e-01, -2.80929171e-01],
       [ 1.26054105e+00, -6.44254234e-01, -1.58601943e-01],
       [ 4.79804048e-01, -6.43749468e-01, -1.28564899e-01],
       [ 1.07035593e+00, -6.97842314e-01, -1.65293598e-01],
       [ 8.58572525e-01, -5.59774876e-01, -1.03705833e-01],
       [-1.52324362e+00,  1.40873753e+00,  6.71391921e+00],
       [-1.44338677e+00, -9.44685200e-01, -2.46022054e-01],
       [-1.44323931e+00, -9.44521418e-01, -2.45982822e-01],
       [ 1.26050223e+00, -6.44294168e-01, -1.58611311e-01],
       [ 1.26969300e+00, -6.34517000e-01, -1.56302178e-01],
       [ 6.88549926e-01, -8.24229970e-01, -2.36213868e-01],
       [-1.44338842e+00, -9.44687072e-01, -2.46022477e-01],
       [ 2.31629575e+00,  6.14345302e-01

In [73]:
# Create a DataFrame with the principal components data
col_names = [f"PC {i}" for i in range(1,comp_num + 1)]
pcs_df = pd.DataFrame(components, columns=col_names, index=crypto_df.index)
print(pcs_df.shape)
pcs_df.head(20)

(133, 3)


,PC 1,PC 2,PC 3
42,-0.904683,-0.982262,-0.280979
NSR,-1.285198,0.575793,-0.647674
TRI,-1.481217,-0.802899,-0.302799
CMTC,0.688329,-0.824450,-0.236262
CHAT,-0.904453,-0.982034,-0.280929
QRL,1.260541,-0.644254,-0.158602
PURA,0.479804,-0.643749,-0.128565
BTCP,1.070356,-0.697842,-0.165294
ADK,0.858573,-0.559775,-0.103706
DAPS,-1.523244,1.408738,6.713919


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [74]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.plot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [75]:
# Initialize the K-Means model
k=4
model = KMeans(n_clusters=k, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df["Class"] = model.labels_
clustered_df=pd.concat([crypto_df, pcs_df], join='inner', axis=1)
clustered_df["CoinName"] = coinname["CoinName"]
clustered_df

,Algorithm,ProofType,TotalCoinsMined,MaxSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,41.999952,42,-0.904683,-0.982262,-0.280979,1,42 Coin
NSR,PoS,PoS,6173556217.8311,0,-1.285198,0.575793,-0.647674,1,NuShares
TRI,X13,PoW/PoS,191620.842403,0,-1.481217,-0.802899,-0.302799,1,Triangles Coin
CMTC,Scrypt,PoW,872830,0,0.688329,-0.824450,-0.236262,1,CometCoin
CHAT,Scrypt,PoW/PoS,1000000000,-1,-0.904453,-0.982034,-0.280929,1,OpenChat
...,...,...,...,...,...,...,...,...,...
NAV,X13,PoW/PoS,72926127.614232,-1,-1.481201,-0.802883,-0.302795,1,NavCoin
EOS,DPoS,DPoS,1048114190.7852,-1,-1.812942,3.741973,-2.312350,2,EOS
NVC,Scrypt,PoW/PoS,3597699.002914,-1,-0.904682,-0.982261,-0.280979,1,NovaCoin
NXT,PoS,PoS/LPoS,998999927.937691,1000000000,-1.152840,0.456101,-0.518230,1,Nxt


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [76]:
# Scale data to create the scatter plot
mm_scaler = MinMaxScaler()
plot_data = mm_scaler.fit_transform(
    clustered_df[["MaxSupply", "TotalCoinsMined"]]
)
plot_df = pd.DataFrame(
    plot_data, columns=["MaxSupply", "TotalCoinsMined"], index=clustered_df.index
)
plot_df["Class"] = clustered_df["Class"]
plot_df["CoinName"] = clustered_df["CoinName"]

plot_df.head(20)

,MaxSupply,TotalCoinsMined,Class,CoinName
42,2.047619e-12,0.000000e+00,1,42 Coin
NSR,4.761905e-14,3.250900e-04,1,NuShares
TRI,4.761905e-14,1.008825e-08,1,Triangles Coin
CMTC,4.761905e-14,4.595968e-08,1,CometCoin
CHAT,0.000000e+00,5.265847e-05,1,OpenChat
QRL,5.000000e-06,4.003679e-06,1,Quantum Resistant Ledger
PURA,0.000000e+00,9.918694e-06,1,Pura
BTCP,1.089219e-06,2.010941e-07,1,Bitcoin Private
ADK,4.761905e-14,1.316460e-06,1,Aidos Kuneen
DAPS,3.333333e-03,3.281648e-03,0,DAPS Coin


In [77]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
plot_df.hvplot.scatter(
    x="TotalCoinsMined", y="MaxSupply", hover_cols=["CoinName"], by="Class"
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (MaxSupply,CoinName)

#### Table of Tradable Cryptocurrencies

In [78]:
# Table with tradable cryptos
clustered_df[
    [
        "CoinName",
        "Algorithm",
        "ProofType",
        "MaxSupply",
        "TotalCoinsMined",
        "Class",
    ]
].hvplot.table()

:Table   [CoinName,Algorithm,ProofType,MaxSupply,TotalCoinsMined,Class]

In [79]:
# Print the total number of tradable cryptocurrencies
print(f"There are {clustered_df.shape[0]} tradable cryptocurrencies.")

There are 133 tradable cryptocurrencies.
